In [ ]:
!pip install nltk
# import nltk
# nltk.download('wordnet')


In [ ]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import re
import os
import pickle

# 1 加载数据

In [ ]:
# 加载数据
train = pd.read_csv('./data/train.csv', sep='\t')
train = train.sample(frac=1)  # 打乱顺序
test = pd.read_csv('./data/test.csv', sep='\t')
print('Train shape:', train.shape)
print('Test shape:', test.shape)


# 2 文本处理

In [ ]:
from nltk.stem import WordNetLemmatizer
import re
def preprocess_text(document):
    stemmer = WordNetLemmatizer()
    # 删除符号
    text = str(document)
    text = text.replace("\n", ' ')
    # 用单个空格替换多个空格
    text = re.sub(r'\s+', ' ', text, flags=re.I)

    # 转换为小写
    text = text.lower()

    # 词形还原
    tokens = text.split()
    tokens = [stemmer.lemmatize(word) for word in tokens]

    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

In [ ]:

train["title"] = train["title"].progress_apply(lambda x: preprocess_text(x))
train["abstract"] = train["abstract"].progress_apply(lambda x: preprocess_text(x))
test["title"] = test["title"].progress_apply(lambda x: preprocess_text(x))
test["abstract"] = test["abstract"].progress_apply(lambda x: preprocess_text(x))


# 3 训练集合成

In [ ]:
train_text_list = []
for index, row in train.iterrows():
    title = row["title"]
    abstract = row["abstract"]
    text = "[CLS] " + title + " [SEP] " + abstract + " [SEP]"
    train_text_list.append(text)
train['text'] = train_text_list

In [ ]:
label_path = "data/label_id2cate.pkl"
#将标签进行转换
label_id2cate = dict(enumerate(train.categories.unique()))
label_cate2id = {value: key for key, value in label_id2cate.items()}

with open(label_path, 'wb') as f:
    pickle.dump(label_id2cate, f, pickle.HIGHEST_PROTOCOL)


In [ ]:

train['label'] = train['categories'].map(label_cate2id)
train_data = pd.DataFrame(columns=["text","label"])
train_data["text"] = train["text"]
train_data["label"] = train["label"]


In [ ]:
train_data.to_csv('data/train_clean_data.csv', sep='\t', index=False)

# 4 测试集合成

In [ ]:
test_text_list = []
for index, row in test.iterrows():
    title = row["title"]
    abstract = row["abstract"]
    text = "[CLS] " + title + " [SEP] " + abstract + " [SEP]"
    test_text_list.append(text)
test['text'] = test_text_list
test['label'] = [-1]*len(test) 
# 去除换行符
test_data = test.drop(['paperid', 'title', 'abstract'], axis=1, inplace=False)

In [ ]:
test_data.to_csv('data/p_data_5w/test_clean_data.csv', sep='\t', index=False)
